In [1]:
import glob
import math
import sys
import time
from pathlib import Path
from typing import Optional, Tuple, Union
import math
import lightning as L
import torch
from lightning.fabric.strategies import FSDPStrategy, XLAStrategy
from torch.utils.data import DataLoader
from functools import partial
# support running without installing as a package
# wd = Path(__file__).parent.parent.resolve()
# sys.path.append(str(wd))
# from apex.optimizers import FusedAdam #torch optimizer has a cuda backend, which is faster actually
from lit_gpt.model import GPT, Block, Config, CausalSelfAttention
from lit_gpt.packed_dataset import CombinedDataset, PackedDataset
from lit_gpt.speed_monitor import SpeedMonitorFabric as Monitor
from lit_gpt.speed_monitor import estimate_flops, measure_flops
from lit_gpt.utils import chunked_cross_entropy, get_default_supported_precision, num_parameters, step_csv_logger, lazy_load
from pytorch_lightning.loggers import WandbLogger
from lit_gpt import FusedCrossEntropyLoss
import random

/home/aiops/wangsd/miniforge3/envs/TL/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/aiops/wangsd/miniforge3/envs/TL/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/aiops/wangsd/miniforge3/envs/TL/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/home/aiops/wangsd/miniforge3/envs/TL/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/aiops/wangsd/miniforge3/envs/TL did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/aiops/wangsd/miniforge3/envs/TL/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/home/aiops/wangsd/miniforge3/etc/xml/catalog file'), PosixPath('/etc/xml/catalog'), PosixPath('file')}
  warn(msg)
/home/aiops/wangsd/miniforge3/envs/TL/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/aiops/wangsd/miniforge3/envs/TL/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarnin

In [2]:
# Load Checkpoint

model_name = "tiny_LLaMA_120M_AttnSSM"
name = model_name
out_dir = Path("out") / name
version = 4

# Hyperparameters
num_of_devices = 2
global_batch_size = 16
learning_rate = 1e-3
# micro_batch_size = 16
micro_batch_size = 4
max_step = 715256
warmup_steps = 1000
log_step_interval = 100
eval_iters = 100
save_step_interval = 20000
eval_step_interval = 2000


weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0
decay_lr = True
min_lr = 4e-5

batch_size = global_batch_size // num_of_devices
gradient_accumulation_steps = batch_size // micro_batch_size
assert gradient_accumulation_steps > 0
warmup_iters = warmup_steps * gradient_accumulation_steps




max_iters = max_step * gradient_accumulation_steps
lr_decay_iters = max_iters
log_iter_interval = log_step_interval * gradient_accumulation_steps


In [3]:
train_data_dir = Path("/home/aiops/wangsd/TinyLlama/data/mix_sample_combined")
val_data_dir = Path("/home/aiops/wangsd/TinyLlama/data/mix_sample_combined")

In [4]:
# Treat all dataset equally by their size. If you want to use a different weight for a dataset, add it to the list with the weight.
train_data_config = [
    ("train_ind", 1.0),
]

val_data_config = [
    ("train_ind", 1.0),
]

hparams = {k: v for k, v in locals().items() if isinstance(v, (int, float, str)) and not k.startswith("_")}
# logger = step_csv_logger("out", name, flush_logs_every_n_steps=log_iter_interval)

import os
import wandb
def read_key_from_file(file_path):
    with open(file_path, 'r') as file:
        return file.read().strip()
key_file_path = '/home/aiops/wangsd/TinyLlama_3/wandb_key.txt'
wandb_key = read_key_from_file(key_file_path)
wandb.login(key=wandb_key)
# wandb_logger = WandbLogger(name="tiny_llama_120M_SSM_O2_exp", id="tiny_llama_120M_SSM_O2_exp", project="Rebuttal")
# wandb_logger = WandbLogger(name=f"{model_name}_v{version}", id=f"{model_name}_v{version}", project="TL3")


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: sanderwang. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/aiops/wangsd/.netrc


True

In [5]:
precision = None
tpu = False
devices = 1

precision = precision or get_default_supported_precision(training=True, tpu=tpu)

if devices > 1:
    if tpu:
        # For multi-host TPU training, the device count for Fabric is limited to the count on a single host.
        devices = "auto"
        strategy = XLAStrategy(sync_module_states=False)
    else:
        strategy = FSDPStrategy(
            auto_wrap_policy={Block},
            activation_checkpointing_policy=None,
            state_dict_type="full",
            limit_all_gathers=True,
            cpu_offload=False,
        )
else:
    strategy = "auto"

# fabric = L.Fabric(devices=devices, strategy=strategy, precision=precision, loggers=[logger, wandb_logger])
# fabric = L.Fabric(devices=devices, strategy=strategy, precision=precision, loggers=[wandb_logger])
fabric = L.Fabric(devices=devices, strategy=strategy, precision=precision, loggers=[])
fabric.print(hparams)


Using bfloat16 Automatic Mixed Precision (AMP)


{'model_name': 'tiny_LLaMA_120M_AttnSSM', 'name': 'tiny_LLaMA_120M_AttnSSM', 'version': 4, 'num_of_devices': 2, 'global_batch_size': 16, 'learning_rate': 0.001, 'micro_batch_size': 4, 'max_step': 715256, 'warmup_steps': 1000, 'log_step_interval': 100, 'eval_iters': 100, 'save_step_interval': 20000, 'eval_step_interval': 2000, 'weight_decay': 0.1, 'beta1': 0.9, 'beta2': 0.95, 'grad_clip': 1.0, 'decay_lr': True, 'min_lr': 4e-05, 'batch_size': 8, 'gradient_accumulation_steps': 2, 'warmup_iters': 2000, 'max_iters': 1430512, 'lr_decay_iters': 1430512, 'log_iter_interval': 200}


In [6]:

def create_dataloader(
    batch_size: int, block_size: int, data_dir: Path, fabric, shuffle: bool = True, seed: int = 12345, split="train"
) -> DataLoader:
    datasets = []
    data_config = train_data_config if split == "train" else val_data_config
    for prefix, _ in data_config:
        filenames = sorted(glob.glob(str(data_dir / f"{prefix}*")))
        random.seed(seed)
        random.shuffle(filenames)

        dataset = PackedDataset(
            filenames,
            # n_chunks control the buffer size. 
            # Note that the buffer size also impacts the random shuffle
            # (PackedDataset is an IterableDataset. So the shuffle is done by prefetch a buffer and shuffle the buffer)
            n_chunks=8,
            block_size=block_size,
            shuffle=shuffle,
            seed=seed+fabric.global_rank,
            num_processes=fabric.world_size,
            process_rank=fabric.global_rank,
        )
        datasets.append(dataset)

    if not datasets:
        raise RuntimeError(
            f"No data found at {data_dir}. Make sure you ran prepare_redpajama.py to create the dataset."
        )

    weights = [weight for _, weight in data_config]
    sum_weights = sum(weights)
    weights = [el / sum_weights for el in weights]

    combined_dataset = CombinedDataset(datasets=datasets, seed=seed, weights=weights)

    return DataLoader(combined_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)



In [7]:
def create_dataloaders(
    batch_size: int,
    block_size: int,
    fabric,
    train_data_dir: Path = Path("data/redpajama_sample"),
    val_data_dir: Optional[Path] = None,
    seed: int = 12345,
) -> Tuple[DataLoader, DataLoader]:
    # Increase by one because we need the next word as well
    effective_block_size = block_size + 1
    train_dataloader = create_dataloader(
        batch_size=batch_size,
        block_size=effective_block_size,
        fabric=fabric,
        data_dir=train_data_dir,
        shuffle=True,
        seed=seed,
        split="train"
    )
    val_dataloader = (
        create_dataloader(
            batch_size=batch_size,
            block_size=effective_block_size,
            fabric=fabric,
            data_dir=val_data_dir,
            shuffle=False,
            seed=seed,
            split="validation"
        )
        if val_data_dir
        else None
    )
    return train_dataloader, val_dataloader


In [8]:
# resume
resume = True

monitor = Monitor(fabric, window_size=2, time_unit="seconds", log_iter_interval=log_iter_interval)

if fabric.global_rank == 0:
    out_dir.mkdir(parents=True, exist_ok=True)

config = Config.from_name(model_name)

train_dataloader, val_dataloader = create_dataloaders(
    batch_size=micro_batch_size,
    block_size=config.block_size,
    fabric=fabric,
    train_data_dir=train_data_dir,
    val_data_dir=val_data_dir,
    seed=3412,
)
if val_dataloader is None:
    train_dataloader = fabric.setup_dataloaders(train_dataloader)
else:
    train_dataloader, val_dataloader = fabric.setup_dataloaders(train_dataloader, val_dataloader)

fabric.seed_everything(3412)  # same seed for every process to init model (FSDP)

fabric.print(f"Loading model with {config.__dict__}")
t0 = time.perf_counter()
with fabric.init_module(empty_init=(fabric.world_size > 1)):
    model = GPT(config)
    model.apply(partial(model._init_weights ,n_layer=config.n_layer))


fabric.print(f"Time to instantiate model: {time.perf_counter() - t0:.02f} seconds.")
fabric.print(f"Total parameters {num_parameters(model):,}")

# wandb_logger.watch(model, log="all", log_freq=1000)

model = fabric.setup(model)
optimizer = torch.optim.AdamW(
    model.parameters(), lr=learning_rate, weight_decay=weight_decay, betas=(beta1, beta2), foreach=False
)
# optimizer = FusedAdam(model.parameters(), lr=learning_rate, weight_decay=weight_decay, betas=(beta1, beta2),adam_w_mode=True)
optimizer = fabric.setup_optimizers(optimizer)

state = {"model": model, "optimizer": optimizer, "hparams": hparams, "iter_num": 0, "step_count": 0}

if resume is True:
    resume = sorted(out_dir.glob("*.pth"))[-1]
if resume :
    fabric.print(f"Resuming training from {resume}")
    fabric.load(resume, state)


Seed set to 3412


Loading model with {'org': 'StatNLP-research', 'name': 'tiny_LLaMA_120M_AttnSSM', 'block_size': 2048, 'vocab_size': 32000, 'padding_multiple': 64, 'padded_vocab_size': 32000, 'n_layer': 12, 'n_head': 12, 'n_embd': 768, 'n_ssm': 128, 'order': 2, 'rotary_percentage': 1.0, 'parallel_residual': False, 'bias': False, 'n_query_groups': 12, 'shared_attention_norm': False, '_norm_class': 'FusedRMSNorm', 'norm_eps': 1e-05, '_mlp_class': 'LLaMAMLP', 'intermediate_size': 2048, 'condense_ratio': 1, 'time_mixer': 'attnssm', 'parameterization': 'best', 'retnet_dropout': (0.0,)}


You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


n_head 12
n_query_groups 12
head_size 64
config.n_embd 768
In AttentionSSM, the parameterization used is best
In AttentionSSM, the hidden dimension used is 2304
n_head 12
n_query_groups 12
head_size 64
config.n_embd 768
In AttentionSSM, the parameterization used is best
In AttentionSSM, the hidden dimension used is 2304
n_head 12
n_query_groups 12
head_size 64
config.n_embd 768
In AttentionSSM, the parameterization used is best
In AttentionSSM, the hidden dimension used is 2304
n_head 12
n_query_groups 12
head_size 64
config.n_embd 768
In AttentionSSM, the parameterization used is best
In AttentionSSM, the hidden dimension used is 2304
n_head 12
n_query_groups 12
head_size 64
config.n_embd 768
In AttentionSSM, the parameterization used is best
In AttentionSSM, the hidden dimension used is 2304
n_head 12
n_query_groups 12
head_size 64
config.n_embd 768
In AttentionSSM, the parameterization used is best
In AttentionSSM, the hidden dimension used is 2304
n_head 12
n_query_groups 12
head_s

In [9]:
model.transformer.h[0].attn.attn.weight.shape

torch.Size([2304, 768])

In [10]:
model.transformer.h[0].attn.proj.weight.shape

torch.Size([768, 2304])

In [11]:
model.transformer.h[0].attn

CausalSelfAttentionSSM(
  (attn): Linear(in_features=768, out_features=2304, bias=False)
  (proj): Linear(in_features=2304, out_features=768, bias=False)
  (hyenaSSM): SSM_Hyena(
    (in_proj): Linear(in_features=2304, out_features=4608, bias=False)
    (out_proj): Linear(in_features=2304, out_features=2304, bias=False)
    (D): Linear(in_features=2304, out_features=2304, bias=False)
    (filter_activation): GELU(approximate='none')
    (activation): Identity()
  )
)